<a href="https://colab.research.google.com/github/Rocco000/OncoVision/blob/main/Scripts/ModelsScripts/EvaluateSolutionOnNewData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Evaluate the models on new data and explain its predictions**



Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive') #Connect to Google Drive

Authenticate the user

In [ ]:
from google.colab import auth
from googleapiclient.discovery import build

#To authenticate the user that run the script in order to use the correct path
auth.authenticate_user()
drive_service = build('drive', 'v3')

#Get user information
about = drive_service.about().get(fields='user').execute()
user_email = about['user']['emailAddress']
script_owner = False
path_base_model = path_ga1_model = path_ga2_model = path_ga2_solution = data_path = test_path = similar_disease = ""
path_prefix = ""

if user_email =="rocco.iul2000@gmail.com":
  path_prefix = "/content/drive/MyDrive/SE4AI/Model/"
  path_base_model = "/content/drive/MyDrive/SE4AI/Model/EvaluationFirstApproach/model_parameters.pth"
  path_ga1_model = "/content/drive/MyDrive/SE4AI/Model/EvaluationGAFirstApproach/ModelsConfigurations/best_solution.pth"
  path_ga2_model = "/content/drive/MyDrive/SE4AI/Model/EvaluationGASecondApproach/ModelsConfigurations/best_solution.pth"
  path_ga2_solution = "/content/drive/MyDrive/SE4AI/Model/EvaluationGASecondApproach/BestSolutionGA2.csv"
  test_path = "/content/drive/MyDrive/SE4AI/Data/Datasets/FinalDataset/Test/"
  data_path = "/content/drive/MyDrive/SE4AI/Data/Datasets/DatasetRelatedProject/"
  similar_disease = "/content/drive/MyDrive/SE4AI/Data/Datasets/Dataset1/"
  %run '/content/drive/MyDrive/SE4AI/Scripts/ModelArchitecture1.ipynb'
  %run '/content/drive/MyDrive/SE4AI/Scripts/ModelArchitecture2.ipynb'
  %run '/content/drive/MyDrive/SE4AI/Scripts/Explainability.ipynb'
else:
  path_prefix = "/content/drive/MyDrive/LinkToOncoVision/SE4AI/Model/"
  path_base_model = "/content/drive/MyDrive/LinkToOncoVision/SE4AI/Model/EvaluationFirstApproach/model_parameters.pth"
  path_ga1_model = "/content/drive/MyDrive/LinkToOncoVision/SE4AI/Model/EvaluationGAFirstApproach/ModelsConfigurations/best_solution.pth"
  path_ga2_model = "/content/drive/MyDrive/LinkToOncoVision/SE4AI/Model/EvaluationGASecondApproach/ModelsConfigurations/best_solution.pth"
  path_ga2_solution = "/content/drive/MyDrive/LinkToOncoVision/SE4AI/Model/EvaluationGASecondApproach/BestSolutionGA2.csv"
  test_path = "/content/drive/MyDrive/LinkToOncoVision/SE4AI/Data/Datasets/FinalDataset/Test/"
  data_path = "/content/drive/MyDrive/LinkToOncoVision/SE4AI/Data/Datasets/DatasetRelatedProject/"
  similar_disease = "/content/drive/MyDrive/LinkToOncoVision/SE4AI/Data/Datasets/Dataset1/"
  %run '/content/drive/MyDrive/LinkToOncoVision/SE4AI/Scripts/ModelArchitecture1.ipynb'
  %run '/content/drive/MyDrive/LinkToOncoVision/SE4AI/Scripts/ModelArchitecture2.ipynb'
  %run '/content/drive/MyDrive/LinkToOncoVision/SE4AI/Scripts/Explainability.ipynb'

Import the libraries

In [ ]:
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

import csv

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import torch.nn.functional as F

import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import os

from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ",device)

#Load the datasets

In [ ]:
transform = transforms.Compose([
      transforms.Resize((600, 450)),  # Resize the images to 600x450
      transforms.ToTensor()  # Convert the images to tensors
  ])

#Load the test set
test_data = ImageFolder(root=test_path, transform=transform)
print("Test set labels:")
print(test_data.class_to_idx)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True, num_workers=2, drop_last=False)

#Load the related project dataset
related_project_dataset = ImageFolder(root=data_path, transform=transform)
print("Related project labels:")
print(related_project_dataset.class_to_idx)
related_project_loader = DataLoader(related_project_dataset, batch_size=32, shuffle=True, num_workers=2, drop_last=False)

#Load the bowen disease images
bowen_images_list = list()
print("Bowen images:",len(os.listdir(similar_disease+"bowenDisease/")))
for image_name in os.listdir(similar_disease+"bowenDisease/"):
  image_path = similar_disease+"bowenDisease/"+image_name
  image = Image.open(image_path)
  image = transform(image)
  bowen_images_list.append(image)

bowen_loader = DataLoader(bowen_images_list, batch_size=32, shuffle=True, num_workers=2, drop_last=False)

#Load the carcinoma images
carcinoma_images_list = list()
print("Carcinoma images:",len(os.listdir(similar_disease+"carcinoma/")))
for image_name in os.listdir(similar_disease+"carcinoma/"):
  image_path = similar_disease+"carcinoma/"+image_name
  image = Image.open(image_path)
  image = transform(image)
  carcinoma_images_list.append(image)

carcinoma_loader = DataLoader(carcinoma_images_list, batch_size=32, shuffle=True, num_workers=2, drop_last=False)

#Evaluate the model on new data

In [ ]:
def evaluate_model(model, data_loader, bool_similar_disease):
  true_labels = list()
  predicted_labels = list()

  #To store the images to explainability process
  wrong_images = list()
  correct_images = list()
  wrong_predictions = list()
  correct_predictions = list()

  with torch.no_grad():
    if bool_similar_disease:
      flag = True
      for images in data_loader:
        images = images.to(device)
        predictions = model(images)
        predictions = F.softmax(predictions, dim=1)
        _, predictions = torch.max(predictions, 1)
        predicted_labels.extend(predictions.cpu().numpy())

        if flag:
          correct_images = images[:10]
          flag = False
    else:
      for images, labels in data_loader:
        #Move the image on gpu or cpu. It depends by device variable
        images = images.to(device)
        labels = labels.to(device)

        #Provide the samples to the model
        predictions = model(images)

        #Apply the Softmax activation function. Dim=1 because the output size is [64,2] where the model prediction is in the second column
        predictions = F.softmax(predictions, dim=1)

        #To extract the predicted class with the highest probability for each input sample. 1 to indicate on which dimension apply the max
        _, predictions = torch.max(predictions, 1)

        #Store the true labels and the predicted labels
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predictions.cpu().numpy())

        #Store the images to explainability process
        if len(wrong_images)!=5 or len(correct_images)!=5:
          pred = predictions.cpu().numpy()
          lab = labels.cpu().numpy()

          i = 0
          for p,l in zip(pred,lab):
            if p!=l:
              if len(wrong_images)!=5:
                image = images[i].cpu()
                wrong_images.append(image)
                wrong_predictions.append((l,p))
            else:
              if len(correct_images)!=5:
                image = images[i].cpu()
                correct_images.append(image)
                correct_predictions.append((l,p))

            if len(wrong_images)==5 and len(correct_images)==5:
              break

            i+=1

  true_labels = np.array(true_labels)
  predicted_labels = np.array(predicted_labels)

  if bool_similar_disease:
    return predicted_labels, correct_images
  else:
    #Define the evaluation metrics
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)

    #Define the confusion metrix
    cm = confusion_matrix(true_labels, predicted_labels)
    tn, fp, fn, tp = cm.ravel()
    return cm, tp, tn, fp, fn, accuracy, precision, recall, f1, wrong_images, wrong_predictions, correct_images, correct_predictions

To obtain the input size of the first linear layer in GA2

In [ ]:
def check_validity(solution):
  #Computing the input size of the first nn.Linear

  width_in, height_in, size = size_nn_linear_calculator(layer_type=5, width=450, height=600, channels=None) # first conv2d
  i = 0
  for element in solution:
    if element == 1:
      #conv-128
      #To avoid the presence of 4 consecutive conv0 layers (loss.backward() out of memory)
      if i>2:
        if (solution[i-1]>=1 and solution[i-1]<=5) and (solution[i-2]>=1 and solution[i-2]<=5) and (solution[i-3]>=1 and solution[i-3]<=5):
          return False,0
      elif i==2:
        if (solution[i-1]>=1 and solution[i-1]<=5) and (solution[i-2]>=1 and solution[i-2]<=5):
          return False,0

      if (i-1)>=0:
        j=i-1
        flag=False
        while j>=0 and (not flag):
          if solution[j]==6 or solution[j]==7 or solution[j]==8 or solution[j]==9: #Due to Cuda out of memory, we can't have two conv-128 without a pooling layer
            flag=True
          j=j-1
        if not flag:
          return False,0

      width_in, height_in, size = size_nn_linear_calculator(layer_type=1, width=width_in, height=height_in, channels=None)
    elif element == 2:
      #conv-64
      #To avoid the presence of 4 consecutive conv0 layers (loss.backward() out of memory)
      if i>2:
        if (solution[i-1]>=1 and solution[i-1]<=5) and (solution[i-2]>=1 and solution[i-2]<=5) and (solution[i-3]>=1 and solution[i-3]<=5):
          return False,0
      elif i==2:
        if (solution[i-1]>=1 and solution[i-1]<=5) and (solution[i-2]>=1 and solution[i-2]<=5):
          return False,0

      width_in, height_in, size = size_nn_linear_calculator(layer_type=2, width=width_in, height=height_in, channels=None)
    elif element == 3:
      #conv-32
      #To avoid the presence of 4 consecutive conv0 layers (loss.backward() out of memory)
      if i>2:
        if (solution[i-1]>=1 and solution[i-1]<=5) and (solution[i-2]>=1 and solution[i-2]<=5) and (solution[i-3]>=1 and solution[i-3]<=5):
          return False,0
      elif i==2:
        if (solution[i-1]>=1 and solution[i-1]<=5) and (solution[i-2]>=1 and solution[i-2]<=5):
          return False,0

      width_in, height_in, size = size_nn_linear_calculator(layer_type=3, width=width_in, height=height_in, channels=None)
    elif element == 4:
      #conv-16
      #To avoid the presence of 4 consecutive conv0 layers (loss.backward() out of memory)
      if i>2:
        if (solution[i-1]>=1 and solution[i-1]<=5) and (solution[i-2]>=1 and solution[i-2]<=5) and (solution[i-3]>=1 and solution[i-3]<=5):
          return False,0
      elif i==2:
        if (solution[i-1]>=1 and solution[i-1]<=5) and (solution[i-2]>=1 and solution[i-2]<=5):
          return False,0

      width_in, height_in, size = size_nn_linear_calculator(layer_type=4, width=width_in, height=height_in, channels=None)
    elif element == 5:
      #conv-8
      #To avoid the presence of 4 consecutive conv0 layers (loss.backward() out of memory)
      if i>2:
        if (solution[i-1]>=1 and solution[i-1]<=5) and (solution[i-2]>=1 and solution[i-2]<=5) and (solution[i-3]>=1 and solution[i-3]<=5):
          return False,0
      elif i==2:
        if (solution[i-1]>=1 and solution[i-1]<=5) and (solution[i-2]>=1 and solution[i-2]<=5):
          return False,0

      width_in, height_in, size = size_nn_linear_calculator(layer_type=5, width=width_in, height=height_in, channels=None)
    elif element == 6:
      #max-3
      if (i-1)>=0:
        if solution[i-1]!=12 and solution[i-1]!=13 and solution[i-1]!=1 and solution[i-1]!=2 and solution[i-1]!=3 and solution[i-1]!=4 and solution[i-1]!=5: #if before the pooling layer there isn't a activation layer or a convolutional layer
          return False, 0

      #Find the last convolutional layer before the actual layer to define the number of output channels
      j = i-1
      num_channels = 8 #because our first layer is a conv-8
      flag = False
      while j>=0 and (not flag):
        if solution[j] == 1:
          num_channels = 128
          flag = True
        elif solution[j] == 2:
          num_channels = 64
          flag = True
        elif solution[j] == 3:
          num_channels = 32
          flag = True
        elif solution[j] == 4:
          num_channels = 16
          flag = True
        elif solution[j] == 5:
          num_channels = 8
          flag = True
        j = j-1
      width_in, height_in, size = size_nn_linear_calculator(layer_type=6, width=width_in, height=height_in, channels=num_channels)
    elif element == 7:
      #max-2
      if (i-1)>=0:
        if solution[i-1]!=12 and solution[i-1]!=13 and solution[i-1]!=1 and solution[i-1]!=2 and solution[i-1]!=3 and solution[i-1]!=4 and solution[i-1]!=5: #if before the pooling layer there isn't a activation layer or a convolutional layer
          return False, 0

      #Find the last convolutional layer before the actual layer to define the number of output channels
      j = i-1
      num_channels = 8 #because our first layer is a conv-8
      flag = False
      while j>=0 and (not flag):
        if solution[j] == 1:
          num_channels = 128
          flag = True
        elif solution[j] == 2:
          num_channels = 64
          flag = True
        elif solution[j] == 3:
          num_channels = 32
          flag = True
        elif solution[j] == 4:
          num_channels = 16
          flag = True
        elif solution[j] == 5:
          num_channels = 8
          flag = True
        j = j-1
      width_in, height_in, size = size_nn_linear_calculator(layer_type=7, width=width_in, height=height_in, channels=num_channels)
    elif element == 8:
      #avg-3
      if (i-1)>=0:
        if solution[i-1]!=12 and solution[i-1]!=13 and solution[i-1]!=1 and solution[i-1]!=2 and solution[i-1]!=3 and solution[i-1]!=4 and solution[i-1]!=5: #if before the pooling layer there isn't a activation layer or a convolutional layer
          return False, 0

      #Find the last convolutional layer before the actual layer to define the number of output channels
      j = i-1
      num_channels = 8 #because our first layer is a conv-8
      flag = False
      while j>=0 and (not flag):
        if solution[j] == 1:
          num_channels = 128
          flag = True
        elif solution[j] == 2:
          num_channels = 64
          flag = True
        elif solution[j] == 3:
          num_channels = 32
          flag = True
        elif solution[j] == 4:
          num_channels = 16
          flag = True
        elif solution[j] == 5:
          num_channels = 8
          flag = True
        j = j-1
      width_in, height_in, size = size_nn_linear_calculator(layer_type=8, width=width_in, height=height_in, channels=num_channels)
    elif element == 9:
      #avg-2
      if (i-1)>=0:
        if solution[i-1]!=12 and solution[i-1]!=13 and solution[i-1]!=1 and solution[i-1]!=2 and solution[i-1]!=3 and solution[i-1]!=4 and solution[i-1]!=5: #if before the pooling layer there isn't a activation layer or a convolutional layer
          return False, 0

      #Find the last convolutional layer before the actual layer to define the number of output channels
      j = i-1
      num_channels = 8 #because our first layer is a conv-8
      flag = False
      while j>=0 and (not flag):
        if solution[j] == 1:
          num_channels = 128
          flag = True
        elif solution[j] == 2:
          num_channels = 64
          flag = True
        elif solution[j] == 3:
          num_channels = 32
          flag = True
        elif solution[j] == 4:
          num_channels = 16
          flag = True
        elif solution[j] == 5:
          num_channels = 8
          flag = True
        j = j-1
      width_in, height_in, size = size_nn_linear_calculator(layer_type=9, width=width_in, height=height_in, channels=num_channels)
    elif element == 10:
      #Dropout2d
      if (i-1)>=0:
        if solution[i-1]!=6 and solution[i-1]!=7 and solution[i-1]!=8 and solution[i-1]!=9: #If before the dropout layer there isn't a pooling layer.
          return False, 0
      else:
        return False, 0
    elif element == 11:
      #BatchNorm
      if (i-1)>=0:
        if solution[i-1]!=1 and solution[i-1]!=2 and solution[i-1]!=3 and solution[i-1]!=4 and solution[i-1]!=5:
          return False, 0
    elif element == 12 or element == 13:
      #ReLU and LeakyReLU
      if (i-1)>=0:
        if solution[i-1]!=1 and solution[i-1]!=2 and solution[i-1]!=3 and solution[i-1]!=4 and solution[i-1]!=5 and solution[i-1]!=11: #if before the activation layer there isn't a convolutional layer
          return False, 0

    i = i+1

  size = int(size)
  if size<8 or size>25000:
    return False, 0
  else:
    return True, size

Choose the model

In [ ]:
choose = int(input("What model do you want to evaluate? (1 = base model, 2 = GA1, 3 = GA2)\n"))

path_suffix_test = path_suffix_related = path_suffix_csv = path_suffix_carcinoma = path_suffix_bowen= path_explainability = ""
cm =  tp = tn = fp = fn = accuracy = precision = recall = f1 = None

my_model = None
layers = list() #To store the architecture of ga2 solution

match(choose):
  case 1:
    #Base model
    print("We will evaluate the BASE model")
    my_model = ConvModel1()
    my_model.load_state_dict(torch.load(path_base_model))
    my_model = my_model.to(device)
    my_model.eval()

    path_suffix_related = "EvaluationFirstApproach/InVivoExperiment/ConfusionMatrixOnDBRelatedProject.png"
    path_suffix_csv = "EvaluationFirstApproach/InVivoExperiment/EvaluationOnDBRelatedProject.csv"
    path_suffix_carcinoma="EvaluationFirstApproach/InVivoExperiment/PredictionsOnCarcinoma.png"
    path_suffix_bowen="EvaluationFirstApproach/InVivoExperiment/PreditionsOnBowenDisease.png"
    path_explainability = "EvaluationFirstApproach/Explainability/"
  case 2:
    #Best solution GA1
    print("We will evaluate the best solution of GA1")
    my_model = ConvModel1()
    my_model.load_state_dict(torch.load(path_ga1_model))
    my_model = my_model.to(device)
    my_model.eval()

    path_suffix_test = "EvaluationGAFirstApproach/ConfusionMetrixOnTestSet.png"
    path_suffix_related = "EvaluationGAFirstApproach/InVivoExperiment/ConfusionMatrixOnDBRelatedProject.png"
    path_suffix_csv = "EvaluationGAFirstApproach/InVivoExperiment/EvaluationOnDBRelatedProject.csv"
    path_suffix_carcinoma="EvaluationGAFirstApproach/InVivoExperiment/PredictionsOnCarcinoma.png"
    path_suffix_bowen="EvaluationGAFirstApproach/InVivoExperiment/PreditionsOnBowenDisease.png"
    path_explainability = "EvaluationGAFirstApproach/Explainability/"
  case 3:
    #Best solution GA2
    print("We will evaluate the best solution of GA2")

    #Take the architecture of ga2 best solution
    with open(path_ga2_solution, "r", newline="") as csvfile:
      reader = csv.reader(csvfile)
      next(reader) #Jump the header
      i = 0
      for row in reader:
        for element in row:
          if i>=4 and i<=19:
            layers.append(int(float(element)))

          i+=1

    print("GA2 best solution layers: ",layers)
    flag, size = check_validity(layers)
    print("Flag: ",flag)
    print("Input size of the output linear layer: ",size)
    my_model = ConvModel2(layers,size)
    my_model.load_state_dict(torch.load(path_ga2_model))
    my_model = my_model.to(device)
    my_model.eval()

    path_suffix_test = "EvaluationGASecondApproach/ConfusionMetrixOnTestSet.png"
    path_suffix_related = "EvaluationGASecondApproach/InVivoExperiment/ConfusionMatrixOnDBRelatedProject.png"
    path_suffix_csv = "EvaluationGASecondApproach/InVivoExperiment/EvaluationOnDBRelatedProject.csv"
    path_suffix_carcinoma="EvaluationGASecondApproach/InVivoExperiment/PredictionsOnCarcinoma.png"
    path_suffix_bowen="EvaluationGASecondApproach/InVivoExperiment/PreditionsOnBowenDisease.png"
    path_explainability = "EvaluationGASecondApproach/Explainability/"
  case _:
    print("Input error")

if choose>=1 and choose<=3:

  print("Started the evaluation process on the TEST SET...")
  cm, tp, tn, fp, fn, accuracy, precision, recall, f1, wrong_images, wrong_predictions, correct_images, correct_predictions = evaluate_model(my_model, test_loader, False)

  plot_path = path_prefix+path_suffix_test
  plt.figure(figsize=(8, 6))
  sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
  plt.title("Confusion Matrix TEST SET")
  plt.xlabel("Predicted")
  plt.ylabel("True")
  if choose == 2 or choose == 3:
    plt.savefig(plot_path)
  plt.show()

  #Explainability on wrong images
  print("\n\n\nExplainability on WRONG predictions (test set)")

  if choose == 3:
    explain_prediction(my_model, wrong_images, wrong_predictions, 1, path_prefix+path_explainability+"Test/", layers) #passa la configurazione!
  else:
    explain_prediction(my_model, wrong_images, wrong_predictions, 1, path_prefix+path_explainability+"Test/", None)

  #Explainability on correct images
  print("\n Explainability on CORRECT predictions (test set)")
  if choose == 3:
    explain_prediction(my_model, correct_images, correct_predictions, 0, path_prefix+path_explainability+"Test/", layers) #passa la configurazione!
  else:
    explain_prediction(my_model, correct_images, correct_predictions, 0, path_prefix+path_explainability+"Test/", None)

  print("\n\n\nStarted the evaluation process on the RELATED PROJECT DATASET...")
  predictions = labels = None
  cm, tp, tn, fp, fn, accuracy, precision, recall, f1, wrong_images, wrong_predictions, correct_images, correct_predictions = evaluate_model(my_model, related_project_loader, False)

  #Plot the confusion metrix of related project dataset
  plot_path = path_prefix+path_suffix_related
  plt.figure(figsize=(8, 6))
  sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
  plt.title("Confusion Matrix Related Project")
  plt.xlabel("Predicted")
  plt.ylabel("True")
  plt.savefig(plot_path)
  plt.show()

  evaluation_path = path_prefix+path_suffix_csv
  with open(evaluation_path,"w",newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["accuracy","precision","recall","f1","TP","TN","FP","FN"])
    writer.writerow([accuracy,precision,recall,f1,tp,tn,fp,fn])

  #Explainability

  #Explainability on wrong images
  print("\n\n\nExplainability on WRONG predictions (related project dataset)")
  if choose == 3:
    explain_prediction(my_model, wrong_images, wrong_predictions, 1, path_prefix+path_explainability+"DBRelatedProject/", layers) #passa la configurazione!
  else:
    explain_prediction(my_model, wrong_images, wrong_predictions, 1, path_prefix+path_explainability+"DBRelatedProject/", None)


  #Explainability on correct images
  print("\n Explainability on CORRECT predictions (related project dataset)")
  if choose == 3:
    explain_prediction(my_model, correct_images, correct_predictions, 0, path_prefix+path_explainability+"DBRelatedProject/", layers) #passa la configurazione!
  else:
    explain_prediction(my_model, correct_images, correct_predictions, 0, path_prefix+path_explainability+"DBRelatedProject/", None)

  print("\n\n\nEvaluate the model on Bowen disease images...")
  predictions, images_list  = evaluate_model(my_model, bowen_loader, True)
  class_counts = np.bincount(predictions, minlength=2)

  bowen_path=path_prefix+path_suffix_bowen
  plt.figure(figsize=(7, 5))
  plt.bar(["benign","melanoma"], class_counts)
  plt.xlabel('Predicted classes')
  plt.ylabel('Count')
  plt.title('Predicted Class Distribution (Bowen disease)')
  plt.xticks(rotation=45)
  plt.tight_layout()
  plt.savefig(bowen_path)
  plt.show()

  print("\n\nExplainability on Bowen disease images")

  if choose == 3:
    explain_prediction(my_model, images_list, predictions, 2, path_prefix+path_explainability+"Bowen/", layers) #passa la configurazione!
  else:
    explain_prediction(my_model, images_list, predictions, 2, path_prefix+path_explainability+"Bowen/", None)

  print("\n\n\nEvaluate the model on Carcinoma disease images...")
  predictions, images_list = evaluate_model(my_model, carcinoma_loader, True)
  class_counts = np.bincount(predictions, minlength=2)

  carcinoma_path=path_prefix+path_suffix_carcinoma
  plt.figure(figsize=(7, 5))
  plt.bar(["benign","melanoma"], class_counts)
  plt.xlabel('Predicted classes')
  plt.ylabel('Count')
  plt.title('Predicted Class Distribution (Carcinoma)')
  plt.xticks(rotation=45)
  plt.tight_layout()
  plt.savefig(carcinoma_path)
  plt.show()

  print("\n\nExplainability on Carcinoma images")
  if choose == 3:
    explain_prediction(my_model, images_list, predictions, 2, path_prefix+path_explainability+"Carcinoma/", layers) #passa la configurazione!
  else:
    explain_prediction(my_model, images_list, predictions, 2, path_prefix+path_explainability+"Carcinoma/", None)
